# Langchain integration
In a world where the shlux LLM we built is good enough for use, langchain proves to be a very convenient tool for interacting with the model. Langchain has allowed us to easily create a chatbot that can interact with the model. 

In [1]:
from langchain_core.language_models.llms import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from typing import Any, List, Optional
import sys

sys.path.append("../")
from gpt_model.model import GPT
from pretraining_model.load_weights_into_gpt import load_weights_into_gpt
from model_generate import text_to_tokens, generate, tokens_to_text
from pretraining_model.gpt_weights_download import download_and_load_gpt2


import tiktoken
import torch


class ShluxGPT(LLM):
    """
    Custom LLM class.
    """
    model: Any = None
    tokenizer: Any = None

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.model = GPT({
            "vocab_size": 50257,
            "n_embd": 768,
            "n_heads": 12,
            "n_layers": 12,
            "dropout": 0.1,
            "context_length": 1024,
            "bias": True
        })
        settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt2")
        load_weights_into_gpt(self.model, params)
        self.tokenizer = tiktoken.encoding_for_model("gpt2")

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        token_ids = generate(
            model=self.model,
            idx=text_to_tokens(prompt, self.tokenizer),
            max_new_tokens=15,
            context_size=256,
            top_k=25,
            temperature=1.4,
        )
        return tokens_to_text(token_ids, self.tokenizer)

    @property
    def _llm_type(self) -> str:
        return "shluxgpt"

2024-10-02 01:03:38.261896: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 01:03:38.356587: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 01:03:38.385787: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 01:03:38.534923: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-02 01:03:39.762616: W tensorflow/compiler/tf2

In [2]:
llm = ShluxGPT(n=5)
print(llm.invoke("Hello, how are you?"))


checkpoint: 100%|██████████| 77.0/77.0 [00:00<00:00, 286kiB/s]
encoder.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 1.52MiB/s]
hparams.json: 100%|██████████| 90.0/90.0 [00:00<00:00, 233kiB/s]
model.ckpt.data-00000-of-00001: 100%|██████████| 498M/498M [00:28<00:00, 17.2MiB/s] 
model.ckpt.index: 100%|██████████| 5.21k/5.21k [00:00<00:00, 160kiB/s]
model.ckpt.meta: 100%|██████████| 471k/471k [00:00<00:00, 830kiB/s] 
vocab.bpe: 100%|██████████| 456k/456k [00:00<00:00, 800kiB/s] 
2024-10-02 01:04:20.389298: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 154389504 exceeds 10% of free system memory.


Hello, how are you? [laughs]

[laughing sound]

And now you


In [3]:
print(llm.invoke("what do you know?"))

what do you know? I can't remember how it happened... I mean... I was watching someone
